In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.utils import to_categorical

In [2]:
def load_images_and_masks(satellite_folder, mask_folder):
    satellite_images = []
    masks = []
    for filename in os.listdir(satellite_folder):
        sat_img = Image.open(os.path.join(satellite_folder, filename))
        mask_img = Image.open(os.path.join(mask_folder, filename))  # Assurez-vous que les noms correspondent

        satellite_images.append(np.array(sat_img))
        masks.append(np.array(mask_img))

    return np.array(satellite_images), np.array(masks)


In [3]:
color_to_label = {
    (180, 180, 180): 0,  # Gris - Végétation clairsemée
    (0, 100, 0): 1,     # Vert - Couverture arborée
    (255, 187, 34): 2,  # Orange - Broussailles
    (255, 255, 76): 3,  # Jaune - Prairie
    (240, 150, 255): 4, # Rose - Terre cultivée
    (250, 0, 0): 5,     # Rouge - Zones urbanisées
    (240, 240, 240): 6, # Blanc - Neige et glace
    (0, 100, 200): 7,   # Bleu - Plans d'eau permanents
    (0, 150, 160): 8,   # Vert-bleu - Marais herbacé
    (0, 207, 117): 9,   # Vert mangrove - Mangroves
    (250, 230, 160): 10 # Mousse - Lichens
}

In [4]:
path_satellite_images = 'Esa_tile1_Sat'  # Chemin vers les images satellites
path_labelled_images = 'Esa_tile1_resize'  # Chemin vers les images labelisées (masques)

X, Y = load_images_and_masks(path_satellite_images, path_labelled_images)
Y = to_categorical(Y)  # Convertir les masques en catégories

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


FileNotFoundError: [Errno 2] No such file or directory: 'Esa_tile1_resize\\ESA_WorldCover_10m_2021_V200_N30W003_Map_TrueMarble.250m.21600x21600.D2.tif'

In [ ]:
def create_unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Couches descendantes
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)

    # Couches ascendantes
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Remplacer par la forme réelle de vos images et le nombre de canaux (par exemple, (1440, 1440, 3) pour des images RGB)
model = create_unet_model((1440, 1440, 3), num_classes=11)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_test, Y_test))
model.evaluate(X_test, Y_test)